In [4]:
from datetime import datetime
from operator import itemgetter
from os import path

from astropy.table import Table
import numpy as np
from skynet.api.v20 import errors
from skynet.api.v20.methods import download, groups, obs, scopes

In [5]:
group_id = groups.get("Majewski Research Group")["id"]
scope_id = scopes.query(include="*", **{"name": "RRRT"})[0]["id"]
group_id, scope_id

(816, 27)

In [6]:
# fields = "*"
observation_fields = {
    "id": "Observation ID",
    "state": "Observation state",
    "priority": "Priority",
    "name": "Target Name",
    "user.username": "Username",
    "group.name": "Group",
    "raHours": "RA",
    "decDegs": "Dec",
    "currentTelescope.name": "Telescope",
    "teleOwner.name": "Scope Owner",
    "timeIn": "Submitted",
}

observation_criteria = {"groupId": group_id}
# criteria = {"id": 3702107}
observation_results = obs.query(include=",".join(observation_fields.keys()), **observation_criteria)

observations = Table()
for field, col_name in observation_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 1:
            observations[col_name] = list(map(itemgetter(field), observation_results))
        case 2:
            field_dicts = list(map(itemgetter(field_parts[0]), observation_results))
            observations[col_name] = [field_dict[field_parts[1]] if field_dict is not None else "" for field_dict in field_dicts]
observations.write("skynet_observations.csv", overwrite=True)

observations

Observation ID,Observation state,Priority,Target Name,Username,Group,RA,Dec,Telescope,Scope Owner,Submitted
int32,str9,int32,str29,str9,str23,float64,float64,str4,str3,str19
3702107,completed,1,K2-22b,meb4bc,Majewski Research Group,11.2988547222222,2.61905638888889,RRRT,UVA,2019-03-08T19:36:11
7474102,canceled,7,1194_01,mharris,Majewski Research Group,11.188,69.96444444444445,,UVA,2022-03-11T21:48:04
7474104,canceled,7,1251_01,mharris,Majewski Research Group,18.23525,62.85805555555556,,UVA,2022-03-11T22:43:08
7474105,canceled,7,1775_01,mharris,Majewski Research Group,10.007666666666667,39.45777777777778,,UVA,2022-03-11T22:52:48
7484500,completed,7,1775_01,mharris,Majewski Research Group,10.007666666666667,39.45777777777778,RRRT,UVA,2022-03-14T15:18:47
7505658,canceled,7,1251_01,mharris,Majewski Research Group,18.23525,62.85805555555556,RRRT,UVA,2022-03-19T01:24:24
7514494,canceled,7,2017_01,mharris,Majewski Research Group,15.263333333333334,27.361666666666668,RRRT,UVA,2022-03-21T02:09:05
7514505,canceled,7,1295_01,mharris,Majewski Research Group,18.24666666666667,67.87138888888889,RRRT,UVA,2022-03-21T03:22:16
7514506,canceled,7,1273_01,mharris,Majewski Research Group,14.274694444444446,58.390277777777776,,UVA,2022-03-21T03:22:16


In [48]:
exposure_fields = {
    "exps.obsId": "Observation ID",
    "exps.expNum": "Exposure Num",
    "exps.imageId": "Image ID",
    "exps.obs.name": "Name",
    "exps.type": "Type",
    "exps.filterName": "Filter Name",
    "exps.timeTaken": "Time Taken",
    "exps.state": "Image State",
    "exps.expLengthUsed": "Exposure",
    "exps.binningUsed": "Binning",
    "exps.gain": "Gain",
    "exps.camera.name": "Camera",
}

exposure_criteria = {"groupId": group_id}
# exposure_criteria = {"id": 3702107}
# exposure_criteria = {"id": 10408766}

exposure_results = obs.query(include=",".join(exposure_fields.keys()), **exposure_criteria)

exp_structs = [image for result in exposure_results for image in result["exps"]]
images = Table()
for field, col_name in exposure_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 2:
            images[col_name] = list(map(itemgetter(field_parts[1]), exp_structs))
        case 3:
            field_dicts = list(map(itemgetter(field_parts[1]), exp_structs))
            images[col_name] = [field_dict[field_parts[2]] if field_dict is not None else "" for field_dict in field_dicts]
images.write("skynet_images.csv", overwrite=True)

images

Observation ID,Exposure Num,Image ID,Name,Type,Filter Name,Time Taken,Image State,Exposure,Binning,Gain,Camera
int32,int32,str9,str29,str5,str1,object,str8,object,int32,object,str40
3702107,0,r24092276,K2-22b,light,R,2019-03-13T03:29:19,archived,180.0,2,1.20651,UVA SBIG STX-16803 with FLI Filter Wheel
3702107,1,r24092277,K2-22b,light,R,2019-03-13T03:32:29,archived,180.0,2,1.20651,UVA SBIG STX-16803 with FLI Filter Wheel
3702107,2,r24092278,K2-22b,light,R,2019-03-13T03:35:39,archived,180.0,2,1.20651,UVA SBIG STX-16803 with FLI Filter Wheel
3702107,3,r24092279,K2-22b,light,V,2019-03-13T03:39:03,archived,180.0,2,1.20651,UVA SBIG STX-16803 with FLI Filter Wheel
3702107,4,r24092280,K2-22b,light,V,2019-03-13T03:42:13,archived,180.0,2,1.20651,UVA SBIG STX-16803 with FLI Filter Wheel
3702107,5,r24092281,K2-22b,light,V,2019-03-13T03:45:23,archived,180.0,2,1.20651,UVA SBIG STX-16803 with FLI Filter Wheel
3702107,6,r24092282,K2-22b,light,V,2019-03-13T03:48:32,archived,360.0,2,1.20651,UVA SBIG STX-16803 with FLI Filter Wheel
3702107,7,r24092283,K2-22b,light,V,2019-03-13T03:54:41,archived,360.0,2,1.20651,UVA SBIG STX-16803 with FLI Filter Wheel
3702107,8,r24092284,K2-22b,light,V,2019-03-13T04:00:51,archived,360.0,2,1.20651,UVA SBIG STX-16803 with FLI Filter Wheel


In [58]:
foo = [exposure for observation in exposure_results for exposure in observation["exps"]]
print(len(foo))
foo[:14]
# exposure_results[:2]

163868


[{'expLengthUsed': 180.0,
  'obsId': 3702107,
  'state': 'archived',
  'camera': {'name': 'UVA SBIG STX-16803 with FLI Filter Wheel'},
  'timeTaken': '2019-03-13T03:29:19',
  'type': 'light',
  'binningUsed': 2,
  'filterName': 'R',
  'imageId': 'r24092276',
  'expNum': 0,
  'gain': 1.20651,
  'obs': {'name': 'K2-22b'}},
 {'expLengthUsed': 180.0,
  'obsId': 3702107,
  'state': 'archived',
  'camera': {'name': 'UVA SBIG STX-16803 with FLI Filter Wheel'},
  'timeTaken': '2019-03-13T03:32:29',
  'type': 'light',
  'binningUsed': 2,
  'filterName': 'R',
  'imageId': 'r24092277',
  'expNum': 1,
  'gain': 1.20651,
  'obs': {'name': 'K2-22b'}},
 {'expLengthUsed': 180.0,
  'obsId': 3702107,
  'state': 'archived',
  'camera': {'name': 'UVA SBIG STX-16803 with FLI Filter Wheel'},
  'timeTaken': '2019-03-13T03:35:39',
  'type': 'light',
  'binningUsed': 2,
  'filterName': 'R',
  'imageId': 'r24092278',
  'expNum': 2,
  'gain': 1.20651,
  'obs': {'name': 'K2-22b'}},
 {'expLengthUsed': 180.0,
  'ob

In [45]:
images_by_observation = images[images[exposure_fields["exps.state"]] == "archived"].group_by(exposure_fields["exps.obsId"])

# np.min(images_by_observation[exposure_fields["exps.timeTaken"]])

for observation_id, images in zip(images_by_observation.groups.keys, images_by_observation.groups):
    print(observation_id, len(images), np.min(images[exposure_fields["exps.timeTaken"]]), np.max(images[exposure_fields["exps.timeTaken"]]))

Observation ID
--------------
       3702107 12 2019-03-13T03:29:19 2019-03-13T07:52:25
Observation ID
--------------
       7484500 671 2022-03-15T00:24:19 2022-03-16T03:39:52
Observation ID
--------------
       7505658 106 2022-03-19T05:53:54 2022-03-19T09:36:20
Observation ID
--------------
       7514494 804 2022-03-21T03:50:40 2022-03-21T06:58:11
Observation ID
--------------
       7514505 334 2022-03-22T04:52:06 2022-03-22T10:12:52
Observation ID
--------------
       7514508 183 2022-03-25T04:50:09 2022-03-25T09:06:41
Observation ID
--------------
      10067782 268 2023-09-19T03:42:09 2023-09-19T09:59:05
Observation ID
--------------
      10067789 297 2023-09-20T00:15:17 2023-09-20T07:03:55
Observation ID
--------------
      10279223 2 2023-11-11T08:12:16 2023-11-11T08:12:41
Observation ID
--------------
      10286988 1 2023-11-12T02:38:30 2023-11-12T02:38:30
Observation ID
--------------
      10287062 2 2023-11-12T04:58:00 2023-11-12T04:58:49
Observation ID
-------------

In [7]:
for observation in observations:
    obs_id = observation[observation_fields["id"]]
    target = observation[observation_fields["name"]].strip().replace(" ", "_")
    filename = f"{obs_id}-{target}.zip"
    if path.isfile(filename):
        print(f"Observation {obs_id:10d} already downloaded")
        continue
    print(f"Downloading observation {obs_id:10d}...", end=" ")
    start = datetime.now()
    try:
        _, data = download.fits(**{"obs": obs_id})
    except errors.BadRequestError:
        print("skipped - got bad request")
        continue
    filesize = len(data)
    duration = datetime.now() - start
    if filesize > 0:
        with open(filename, "wb") as f:
            f.write(data)
        print(f"got {filesize/1024/1024:7.1f}MB at {filesize / duration.seconds * 8 / 1024 / 1024:5.2f}Mbps saved to {filename}")


Observation    3702107 already downloaded
Observation    7484500 already downloaded
Observation    7505658 already downloaded
Observation    7514494 already downloaded
Observation    7514505 already downloaded
Observation    7514508 already downloaded
Observation   10067782 already downloaded
Observation   10067789 already downloaded
Observation   10279223 already downloaded
Observation   10286988 already downloaded
Observation   10287062 already downloaded
Observation   10287063 already downloaded
Observation   10287064 already downloaded
Observation   10291478 already downloaded
Observation   10291510 already downloaded
Observation   10291512 already downloaded
Observation   10291513 already downloaded
Observation   10349683 already downloaded
Observation   10349685 already downloaded
Observation   10408756 already downloaded
Observation   10408762 already downloaded
Observation   10408763 already downloaded
Observation   10408764 already downloaded
Observation   10408766 already dow